In [ ]:
import numpy as np 
import os
from utils_data import interpolation_glorys_raw_data,interpolation_era5_raw_data
import xarray as xr
# Define a reference grid :

size_grid = "8th"
coef = 2
lon_ref = np.linspace(-180,180,1441*coef)[:-1]
lat_ref = np.linspace(-90,90,721*coef)[:-1]


# Define depth
depth=15

# Define a period of test : 
year = 2019

# Create a directory

folder_data = f"/Odyssey/private/t22picar/data/train_glorys_{depth}_{size_grid}/"
if not os.path.exists(folder_data):
    os.makedirs(folder_data)
    print(f"Le dossier '{folder_data}' a été créé.")


In [3]:
# Interpolation of testing data + add a correction
# SSH 
# Add offset due to different mdt
time_end = "2019-01-10"
time_start = "2019-01-01"

def interpolation_ssh_raw_data(year,depth,folder_data): 

    folder_ssh = "/Odyssey/private/t22picar/data/ssh_L4/"
    file_obs = "SSH_L4_CMEMS_2019.nc" # Actually not glorys 

    maps = xr.open_dataset(folder_ssh+file_obs).sel(time=slice(time_start,time_end))
    maps = maps.rename({"latitude": "lat"})
    maps = maps.rename({"longitude": "lon"})
    maps = maps.rename({"adt": "zos"})

    # Get the list of variable names
    variable_names = list(maps.variables.keys())
    variable_names.remove("time")

    for var in variable_names:
        maps[var] = maps[var].astype(np.float32)

    folder_glorys = f"/Odyssey/private/t22picar/data/glorys_{depth}m/"
    file_glorys = f"glorys_multivar_{depth}m_2019.nc"
    maps_glo = xr.open_dataset(folder_glorys+file_glorys).sel(time=slice(time_start,time_end))

    mean_glorys_ssh = np.nanmean(maps_glo.zos.values)
    mean_sat_ssh = np.nanmean(maps.zos.values)
    offset = mean_glorys_ssh-mean_sat_ssh

    maps.zos.values = maps.zos.values + offset

    print("Interpolation cmems ssh ... ")
    maps = regrid_da(maps)
    print("Interpolation done")

    # save data 
    print("Saving...")
    save_file=folder_data+f"cmems_ssh_{year}.nc"
    print(save_file)
    # Sauvegarder le DataArray en fichier NetCDF
    maps.to_netcdf(save_file)
    print("Saving done")

# SST 

def interpolation_sst_raw_data(year,depth,folder_data):
    folder_sst = "/Odyssey/private/t22picar/data/sst_L4/"
    file_sst = "SST_L4_OSTIA_2019.nc"

    maps = xr.open_dataset(folder_sst+file_sst).sel(time=slice(time_start,time_end))
    maps = maps.rename({"latitude": "lat"})
    maps = maps.rename({"longitude": "lon"})
    maps = maps.rename({"analysed_sst": "thetao"})

    # Get the list of variable names
    variable_names = list(maps.variables.keys())
    variable_names.remove("time")

    for var in variable_names:
        maps[var] = maps[var].astype(np.float32)


    print("Interpolation ostia ssh ... ")
    # Interpolation new grid
    maps = regrid_da(maps)
    maps.thetao.values = maps.thetao.values - 273.15
    print("Interpolation done")


    # save data 
    print("Saving...")
    save_file=folder_data+f"ostia_sst_{year}"+".nc"
    print(save_file)
    # Sauvegarder le DataArray en fichier NetCDF
    maps.to_netcdf(save_file)
    print("Saving done")

def interpolation_era5_raw_data_test(folder_data):

    print("open data")
    folder_era5 = "/Odyssey/private/t22picar/data/era5/"
    file_era5 = f"era5_2019.grib" # Actually not glorys 
    map_era5 = xr.open_dataset(folder_era5+file_era5, engine="cfgrib").sel(time=slice(time_start,time_end))
    map_era5['longitude'] = xr.where(map_era5['longitude'] > 180, map_era5['longitude'] - 360, map_era5['longitude'])
    map_era5 = map_era5.rename({"latitude": "lat"})
    map_era5 = map_era5.rename({"longitude": "lon"})

    # Daily mean wind
    print("computing daily mean era5 ...")
    map_era5 = map_era5.resample(valid_time='1D').mean()
    print("computation daily mean done")
    # Interpolation new grid
    print("Interpolating era5 ...")
    map_era5 = regrid_da(map_era5)
    print("interpolation done")
    #Split into two files
    map_era5 = map_era5.drop("number").drop("step").drop("surface").rename({"valid_time": "time"})

    # Get the list of variable names
    variable_names = list(map_era5.variables.keys())
    variable_names.remove("time")

    for var in variable_names:
        map_era5[var] = map_era5[var].astype(np.float32)
    # save data
    print("Saving file ...")
    save_file=f"era5_2019.nc"

    # Sauvegarder le DataArray en fichier NetCDF
    map_era5.to_netcdf(folder_data+save_file)
    print("Saving done")


In [ ]:
# SSH 
#interpolation_ssh_raw_data(year,depth,folder_data)

# SST 

#interpolation_sst_raw_data(year,depth,folder_data)

#ERA? 
interpolation_era5_raw_data_test(folder_data)

open data
